In [14]:
# what we are studying
model = "LR"
dataset = "census"
sensitive_param = "sex"
os.chdir('./jupyter_analysis')

FileNotFoundError: [Errno 2] No such file or directory: './jupyter_analysis'

In [15]:
import pandas as pd
import os

In [16]:
df = pd.read_csv("../Dataset" + "/" + f"{model}_{dataset}_{sensitive_param}_x100.csv")
df.head()

,solver,penalty,dual,tol,C,fit_intercept,intercept_scaling,max_iteration,multi_class,l1_ratio,...,verbose,warm_start,n_jobs,score,AOD,TPR,FPR,counter,timer,inp
0,lbfgs,l2,False,0.00010,10.000000,False,1.000000,10000,ovr,NaN,...,0,False,NaN,0.805306,0.059893,0.09898,0.020807,0,0.378170,"[1, 1, 0, 0.0001, 10.0, 0, 1.0, 10000, 0, 0, 0..."
1,lbfgs,l2,False,0.00010,10.000000,False,1.000000,10000,ovr,NaN,...,0,False,NaN,0.805306,0.059893,0.09898,0.020807,1,0.640464,"[1, 0, 0, 0.0001, 10.0, 0, 1.0, 10000, 0, 0, 0..."
2,newton-cg,none,False,0.78914,47.719794,False,0.539406,10407,auto,NaN,...,0,True,NaN,0.805306,0.059893,0.09898,0.020807,2,0.825221,"[0, 3, 1, 0.7891397155771552, 47.7197942928364..."
3,lbfgs,l2,False,0.00010,10.000000,False,1.000000,10000,ovr,NaN,...,0,False,NaN,0.805306,0.059893,0.09898,0.020807,3,1.080979,"[1, 0, 0, 0.0001, 10.0, 0, 1.0, 10000, 0, 0, 0..."
4,lbfgs,l2,False,0.00010,10.000000,False,1.000000,10000,multinomial,NaN,...,0,False,NaN,0.805306,0.059893,0.09898,0.020807,4,1.603220,"[1, 0, 0, 0.0001, 10.0, 0, 1.0, 10000, 1, 0, 0..."


In [17]:
print(f"Min score: {df['score'].min()}")
print(f"Max score: {df['score'].max()}")
print(f"Min AOD: {df['AOD'].min()}")
print(f"Max AOD: {df['AOD'].max()}")

Min score: 0.7953568357695615
Max score: 0.810711214838472
Min AOD: 0.0045455724043264
Max AOD: 0.2126840140268309


In [18]:
print(f"Best AOD: {df[df['AOD'] == df['AOD'].min()].iloc[0]}")
print(f"Best accuracy: {df[df['score'] == df['score'].min()].iloc[0]}")

Best AOD: solver                                                       liblinear
penalty                                                             l2
dual                                                              True
tol                                                           0.594069
C                                                            65.504002
fit_intercept                                                    False
intercept_scaling                                             8.778587
max_iteration                                                    10050
multi_class                                                       auto
l1_ratio                                                           NaN
class_weight                                                       NaN
random_state                                                      2019
verbose                                                              0
warm_start                                                       Fa

## Recreating the best model

In [19]:
import ast
import sys
sys.path.append("../subjects/")

import numpy as np
import LogisticRegression
from sklearn.model_selection import train_test_split
from adf_data.census import census_data
from adf_data.credit import credit_data
from adf_data.bank import bank_data
from adf_data.compas import compas_data
os.chdir('../')

In [20]:
inp = ast.literal_eval(df[df['AOD'] == df['AOD'].min()].iloc[0]["inp"])
data = {"census":census_data, "credit":credit_data, "bank":bank_data, "compas": compas_data}
X, Y, input_shape, nb_classes = data[dataset]()
Y = np.argmax(Y, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [21]:
LogisticRegression.logistic_regression(inp, X_train, X_test, y_train, y_test, sensitive_param, dataset_name=dataset, save_model=True)

[2, 1, 1, 0.5940687231770445, 65.50400197384276, 0, 8.77858740660061, 10050, 2, 1, 1, 0, 0, 0, 0]
['liblinear', 'l2', True, 0.5940687231770445, 65.50400197384276, False, 8.77858740660061, 10050, 'auto', 'rand', 'balanced', 'None', 0, False, 'None']
['liblinear', 'l2', True, 0.5940687231770445, 65.50400197384276, False, 8.77858740660061, 10050, 'auto', None, None, 2019, 0, False, None]


/home/normenyu/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


(True,
 LogisticRegression(C=65.50400197384276, dual=True, fit_intercept=False,
                    intercept_scaling=8.77858740660061, max_iter=10050,
                    random_state=2019, solver='liblinear',
                    tol=0.5940687231770445),
 ['liblinear',
  'l2',
  True,
  0.5940687231770445,
  65.50400197384276,
  False,
  8.77858740660061,
  10050,
  'auto',
  None,
  None,
  2019,
  0,
  False,
  None],
 0.8065348237317282,
 array([0, 0, 0, ..., 0, 0, 0]),
 ['solver',
  'penalty',
  'dual',
  'tol',
  'C',
  'fit_intercept',
  'intercept_scaling',
  'max_iteration',
  'multi_class',
  'l1_ratio',
  'class_weight',
  'random_state',
  'verbose',
  'warm_start',
  'n_jobs'])

### The most fair model has the following coeficients

In [22]:
import pickle as pk
with open("./trained_models/logisticRegression_census_sex_liblinear_l2_True_0.5940687231770445_65.50400197384276_False_8.77858740660061_10050            _auto_None_None_2019_0_False_None.pkl", "rb") as model_file:
    model = pk.load(model_file)
    print(model.coef_)
    print(model.intercept_)
    print(model.n_iter_)

[[ 0.08574622  0.06388331 -0.02715659 -0.02135337 -1.12969418 -0.11284579
  -0.42472817 -0.14201319  0.04904803  0.46023246  0.15396227  0.0054096
  -0.01666468]]
0.0
[10050]


In [23]:
inp = ast.literal_eval(df[df['score'] == df['score'].max()].iloc[0]["inp"])
LogisticRegression.logistic_regression(inp, X_train, X_test, y_train, y_test, sensitive_param, dataset_name=dataset, save_model=True)

[2, 1, 1, 0.9769791568842524, 37.917604087606755, 1, 6.512768585723584, 9774, 1, 0, 0, 0, 1, 0, 0]
['liblinear', 'l2', True, 0.9769791568842524, 37.917604087606755, True, 6.512768585723584, 9774, 'multinomial', 'none', 'None', 'None', 1, False, 'None']
['liblinear', 'l2', True, 0.9769791568842524, 37.917604087606755, True, 6.512768585723584, 9774, 'ovr', None, None, 2019, 0, False, None]


/home/normenyu/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


(True,
 LogisticRegression(C=37.917604087606755, dual=True,
                    intercept_scaling=6.512768585723584, max_iter=9774,
                    multi_class='ovr', random_state=2019, solver='liblinear',
                    tol=0.9769791568842524),
 ['liblinear',
  'l2',
  True,
  0.9769791568842524,
  37.917604087606755,
  True,
  6.512768585723584,
  9774,
  'ovr',
  None,
  None,
  2019,
  0,
  False,
  None],
 0.810711214838472,
 array([0, 0, 0, ..., 0, 0, 0]),
 ['solver',
  'penalty',
  'dual',
  'tol',
  'C',
  'fit_intercept',
  'intercept_scaling',
  'max_iteration',
  'multi_class',
  'l1_ratio',
  'class_weight',
  'random_state',
  'verbose',
  'warm_start',
  'n_jobs'])

In [25]:
df[df['score'] == df['score'].max()].iloc[0]

,solver,penalty,dual,tol,C,fit_intercept,intercept_scaling,max_iteration,multi_class,l1_ratio,...,verbose,warm_start,n_jobs,score,AOD,TPR,FPR,counter,timer,inp
1180,liblinear,l2,True,0.976979,37.917604,True,6.512769,9774,ovr,NaN,...,0,False,NaN,0.810711,0.044393,0.064366,0.024419,1312,1380.451135,"[2, 1, 1, 0.9769791568842524, 37.9176040876067..."


### The most accurate model has the following coeficients

In [15]:
with open("./trained_models/logisticRegression_census_sex_liblinear_l2_True_0.5940687231770445_65.50400197384276_False_8.77858740660061_10050            _auto_None_None_2019_0_False_None.pkl", "rb") as model_file:
    model = pk.load(model_file)
    print(model.coef_)
    print(model.intercept_)
    print(model.n_iter_)
    print(model.score(X_test,y_test))

[[ 0.08574622  0.06388331 -0.02715659 -0.02135337 -1.12969418 -0.11284579
  -0.42472817 -0.14201319  0.04904803  0.46023246  0.15396227  0.0054096
  -0.01666468]]
0.0
[10050]
0.8065348237317282


In [14]:
df[df['score'] == df['score'].max()]["AOD"]

1180    0.044393
Name: AOD, dtype: float64